# Metro Cancellations

Creating maps, charts, and summary statistics around LA Metro bus trip cancellations in 2022.

In [ ]:
import datetime

import altair
import altair_saver
import contextily
import geopandas
import matplotlib.pyplot as plt
import pandas
import partridge
import toolz
from mpl_toolkits.axes_grid1 import make_axes_locatable

altair.data_transformers.disable_max_rows()

## Load Data

In [ ]:
# Default to loading data for the prior day, which should be complete.
date = pandas.Timestamp.today(tz="US/Pacific").date()
date = date - datetime.timedelta(days=1)

In [ ]:
# December 2021 shakeup GTFS feed.
feed = partridge.load_geo_feed("gtfs_bus_2021_12.zip")
service_by_date = partridge.read_service_ids_by_date("gtfs_bus_2021_12.zip")
service_by_date = pandas.DataFrame.from_records(
    toolz.concat([(k, s) for s in v] for k, v in service_by_date.items()),
    columns=["date", "service_id"],
)

# Trip updates from the relevant day. Note: service by day includes some late night
# trips on the next date. This probably gets cancellations for those late night trips
# wrong! But for now, ignore as a small fraction of the total trips
updates = pandas.read_parquet(f"trip-updates/trip-updates-{date.isoformat()}.parquet")

## Create a service-by-line chart

In [ ]:
# Service patterns for the day
service = service_by_date[service_by_date.date == date]
# Trips for the service patterns
trips = feed.trips[feed.trips.service_id.isin(service.service_id)]
# Route data
routes = feed.routes.assign(
     route=lambda x: x.route_short_name.combine_first(x.route_long_name)
)[["route_id", "route"]]
# Timing data for trips, which we use to compute scheduled service hours.
trip_timings = (
    feed
    .stop_times
    .groupby("trip_id")
    .agg({"arrival_time": max, "departure_time": min})
)
trip_timings = trip_timings.assign(
    duration=pandas.to_timedelta(
        trip_timings.arrival_time - trip_timings.departure_time,
        unit="s"
    ),
    arrival_time=pandas.to_datetime(date).normalize() + pandas.to_timedelta(trip_timings.arrival_time, unit="s"),
    departure_time=pandas.to_datetime(date).normalize() + pandas.to_timedelta(trip_timings.departure_time, unit="s"),

)

# Merge the datasets into a single trips dataframe with timing, route, and updates data.
# Assume that if a trip does not get an update, it is "scheduled". This assumption may
# not be 100%, but seems to work okay.
trips = trips.merge(trip_timings, left_on="trip_id", right_index=True)
trips = trips.merge(routes, on="route_id", how="left",)
trips = (
    trips.merge(updates, on="trip_id", how="left")
    .assign(
        schedule_relationship=lambda x: x.schedule_relationship.fillna("scheduled")
    )       
)

In [ ]:
# Select a subset of the data for the altair strip plot. At this point,
# "scheduled" without a cancellation is presumed to mean that the trip
# was run, so replace the word for clarity in the plot.
to_chart = (
    trips
    [["departure_time", "route", "schedule_relationship"]]
    .replace("scheduled", "run")
)

# The main chart!
chart = altair.Chart(
    to_chart,
    width=1024
).mark_tick(
    size=10,
).encode(
    x=altair.X(
        "departure_time:T",
        title="Departure Time",
        axis=altair.Axis(orient="top", grid=True)
    ),
    y=altair.Y("route:N", title="Route"),
    color=altair.Color(
        'schedule_relationship:N',
        title="Status",
        scale=altair.Scale(domain=["run", "canceled"], range=["darkgreen", "red"])  
    ),
)

# Empty chart to hack in x labels on the bottom as well as top
other = (
    altair.Chart(to_chart.head(0))
    .mark_tick()
    .encode(
        x=altair.X(
            "departure_time:T",
            title="Departure Time",
            axis=altair.Axis(orient="bottom", grid=True)
        ),
        y=altair.Y("route:N", title="Route")
        
    )
)
layered = altair.layer(chart, other).resolve_axis(x="shared", y="shared")

altair_saver.save(
    layered,
    f"cancellation-reports/metro-cancellations-{date.isoformat()}.png",
)
layered

## Create a map

In [ ]:
# Choose the most common shape_id to use for the map
geoms = geopandas.GeoDataFrame(
    (
        trips.groupby("route")
        .agg({"shape_id": lambda g: g.value_counts().index[0]})
        .reset_index()
        .merge(feed.shapes, how="left", on="shape_id")
    ),
    crs="EPSG:4326",
    geometry="geometry",
)
trips_agg = (
    trips
    .assign(count=1)   # unused column for count
    .groupby(["route", "schedule_relationship"], dropna=False)
    .agg({
        "duration": sum,
        "count": "count",
    })
    .unstack(level=1)
)
# https://stackoverflow.com/questions/45878333/merge-multiindex-columns-together-into-1-level
trips_agg.columns = ['_'.join(col) for col in trips_agg.columns.values]
trips_agg = trips_agg.assign(
    duration_canceled=trips_agg.duration_canceled.fillna(pandas.Timedelta(0)),
    count_canceled=trips_agg.count_canceled.fillna(0).astype(int),
    count_scheduled=trips_agg.count_scheduled.astype(int),
)
trips_agg = trips_agg.assign(
    percent_duration_canceled=(
        100*trips_agg.duration_canceled/
        (trips_agg.duration_canceled + trips_agg.duration_scheduled)
    ),
    percent_trips_canceled=(
        100*trips_agg.count_canceled/
        (trips_agg.count_canceled + trips_agg.count_scheduled)
    ),
)

gdf = geopandas.GeoDataFrame(
    trips_agg.merge(geoms, on="route"),
    crs="EPSG:4326",
    geometry="geometry"
)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(16,16))
ax.axis("off")
ax.set_title(f"Metro Bus Cancellations for {date.strftime('%A, %B %-m')}")
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="2%", pad=0.0)
gdf.to_crs(epsg=3857).plot(
    ax=ax,
    column=gdf.percent_duration_canceled,
    cmap="RdYlGn_r",
    lw=2,
    legend=True,
    cax=cax,
    legend_kwds={"label": "Percent of service hours canceled"}
)
contextily.add_basemap(ax, source=contextily.providers.CartoDB.DarkMatterNoLabels)
fig.savefig(
    f"cancellation-reports/metro-cancellations-map-{date.isoformat()}.png",
    dpi=100,
    bbox_inches="tight",
)

## Summary statistics

In [ ]:
out = (
    trips_agg
    .sort_values("percent_duration_canceled", ascending=False)
    .reset_index()
    .rename(columns={
        "route": "Route",
        "duration_canceled": "Service Hours Canceled",
        "duration_scheduled": "Service Hours Run",
        "count_canceled": "Trips Canceled",
        "count_scheduled": "Trips Run",
        "percent_duration_canceled": "Percent of Service Hours Canceled",
        "percent_trips_canceled": "Percent of Trips Canceled",
    })
)

out.to_csv(
    f"cancellation-reports/metro-cancellations-{date.isoformat()}.csv",
    index=False,
)
out.style.hide_index().format(precision=0)

In [ ]:
gdf.duration_canceled.sum()/(gdf.duration_canceled.sum() + gdf.duration_scheduled.sum())

In [ ]:
gdf.count_canceled.sum()/(gdf.count_canceled.sum() + gdf.count_scheduled.sum())